<a href="https://colab.research.google.com/github/HemanthKumarBodduboina/AI-Projects/blob/main/Resume_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('AIproject')

In [16]:
from google.colab import files

# Upload PDF (it will ask you to choose a file)
uploaded = files.upload()

Saving resume.pdf to resume (1).pdf


In [17]:
!pip install pymupdf

In [ ]:
import fitz  # PyMuPDF

# Open the uploaded resume
pdf_path = "resume.pdf"
doc = fitz.open(pdf_path)

resume_text = ""
for page in doc:
    resume_text += page.get_text("text")

print("Extracted Resume Text:\n")
print(resume_text[:1000])  # print first 1000 chars

In [19]:
import openai

# openai.api_key = "AIproject" # This line is not needed if using the client method

job_desc = """
We are hiring an AI Engineer skilled in NLP, Deep Learning, and MLOps (AWS, Docker, Kubernetes).
"""

prompt = f"""
You are a career advisor.
Compare this resume with the job description.
Return:
1. Match Score (0-100)
2. Missing Skills
3. Resume Improvements

Resume: {resume_text}
Job: {job_desc}
"""

# Create an OpenAI client instance
client = openai.OpenAI()


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Match Score: 60

Missing Skills:
- MLOps experience (AWS, Docker, Kubernetes)
- Specific NLP projects or experience
- Specific Deep Learning projects or experience

Resume Improvements:
1. Add a section highlighting experience with MLOps tools like AWS, Docker, and Kubernetes
2. Include specific NLP and Deep Learning projects or experiences to showcase expertise in those areas
3. Consider obtaining certifications or completing projects related to MLOps, NLP, and Deep Learning to strengthen your profile


In [ ]:
!pip install streamlit pyngrok pymupdf openai

In [13]:
%%writefile app.py
import streamlit as st
import fitz  # PyMuPDF
import openai

# ============ Functions ============
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

def analyze_resume(resume_text, job_desc, api_key):
    # Create an OpenAI client instance within the function
    client = openai.OpenAI(api_key=api_key)

    prompt = f"""
    You are a career advisor.
    Compare this resume with the job description.
    Return:
    1. Match Score (0-100)
    2. Missing Skills
    3. Resume Improvements

    Resume: {resume_text}
    Job: {job_desc}
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo", # gpt-5 is not a valid model name, using gpt-3.5-turbo
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# ============ Streamlit UI ============
st.title("📄 AI Resume Analyzer (GPT-5)")

api_key = st.text_input("Enter your OpenAI API Key:", type="password")

uploaded_file = st.file_uploader("Upload Resume (PDF)", type="pdf")
job_desc = st.text_area("Paste Job Description Here:")

if uploaded_file and job_desc and api_key:
    resume_text = extract_text_from_pdf(uploaded_file)
    with st.spinner("Analyzing..."):
        result = analyze_resume(resume_text, job_desc, api_key)
    st.subheader("📌 Analysis Result")
    st.write(result)

Overwriting app.py


In [14]:
from pyngrok import ngrok
from google.colab import userdata

# Replace 'NGROK_AUTH_TOKEN' with the name you gave your secret in Colab
ngrok.set_auth_token(userdata.get('Ngrock'))

In [15]:
from pyngrok import ngrok

# Kill previous tunnels if any
!pkill streamlit

# Start Streamlit
# public_url = ngrok.connect(port=8501) # Incorrect way to pass port
public_url = ngrok.connect(8501) # Correct way to pass port

print("Streamlit URL:", public_url)
!streamlit run app.py --server.port 8501 &>/dev/null&

Streamlit URL: NgrokTunnel: "https://unscreenable-geoponic-alaina.ngrok-free.dev" -> "http://localhost:8501"
